In [1]:
import pandas as pd
import numpy as np

import re
import os
import string

import spacy

import gensim
from gensim import corpora
from gensim.models import CoherenceModel

# libraries for visualization
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

d:\software\python\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
def gen_sum_df(path):
    files = os.listdir(path)
    df_ls = []
    for file in files:
        df = pd.read_csv(path+file)
        df["date"] = [file[:-4]] * df.shape[0]
        df_ls.append(df)
    df_r = pd.concat(df_ls)
    return df_r
    
df_top5 = gen_sum_df("E:\\2021_Summer\\Research\\twitter_scrape\\tweets_data\\top_5_percent_delay\\")
df_bottom5 = gen_sum_df("E:\\2021_Summer\\Research\\twitter_scrape\\tweets_data\\bottom_5_percent_delay\\")
df = pd.concat([df_top5, df_bottom5])

#remove tweets from official accounts
df = df[~ df['username'].str.contains('metrolinx|gotransit|prestocard|oc_transpo|amawithama|rtsmag|ttcstuart'
                                          '|ttchelps|goturner64|infraontario|transefare|trafficservices|cityburlington|'
                                          'cmo_daily|gonoble164|mytofd|gthatransitinfo|mississauga_cal')]
df

,id,timestamp,coordinates,place,language,retweet,hashtags,mentions,user_id,username,name,replies_count,retweets_count,likes_count,photos,urls,video,link,tweet,date
0,552205507487092736,2015-01-05 15:50:27-05:00,NaN,NaN,en,False,[],"[{'screen_name': 'metrolinx', 'name': 'metroli...",17180800,bernardhellen,Bernard Hellen,0,0,0,[],['http://paper.li/bernardhellen'],0,https://twitter.com/bernardhellen/status/55220...,The Bernard Hellen Daily is out! http://t.co/...,15-01-05
1,552186203509100544,2015-01-05 14:33:44-05:00,NaN,NaN,en,False,"['gotrain', 'miltonon']","[{'screen_name': 'metrolinx', 'name': 'metroli...",30044322,mike_cluett,Councillor Mike Cluett,0,0,0,[],[],0,https://twitter.com/Mike_Cluett/status/5521862...,.@Metrolinx #GOtrain #miltonON Good to see mor...,15-01-05
2,552182458813521920,2015-01-05 14:18:51-05:00,NaN,NaN,en,False,['transitprogress'],[],166617790,jeff_p_harris,Jeff Harris,0,0,0,[],[],0,https://twitter.com/jeff_p_harris/status/55218...,"@Metrolinx #transitprogress a list of ""almosts...",15-01-05
3,552174752379719680,2015-01-05 13:48:14-05:00,NaN,NaN,en,False,"['gotrain', 'miltonon']","[{'screen_name': 'metrolinx', 'name': 'metroli...",972761742,brassangels,Brass Angels Milton,0,0,0,[],['http://www.gotransit.com/public/en/updates/M...,0,https://twitter.com/BrassAngels/status/5521747...,RT @Metrolinx: More #GOtrain service on the #m...,15-01-05
5,552161801761488896,2015-01-05 12:56:46-05:00,NaN,"{'id': '378a442883b05c3c', 'url': 'https://api...",en,False,[],"[{'screen_name': 'metrolinx', 'name': 'metroli...",17136009,via_rail,VIA Rail Canada,0,1,0,[],[],0,https://twitter.com/VIA_Rail/status/5521618017...,@steven_petric @Metrolinx @GOtransit HI Steve:...,15-01-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,1079395040160559104,2018-12-30 10:13:26-05:00,NaN,NaN,en,False,[],[],462564480,92040915k,kevin johnson,1,0,0,[],[],0,https://twitter.com/92040915K/status/107939504...,@PRESTOcard Been a few days waiting. That is w...,18-12-30
81,1079391994865041408,2018-12-30 10:01:20-05:00,NaN,NaN,en,False,[],[],267269778,donvinokumar,Vino Udhayakumar,1,0,0,[],[],0,https://twitter.com/DonVinokumar/status/107939...,@PRESTOcard Yes I tried again this morning and...,18-12-30
83,1079502618722848768,2018-12-30 17:20:55-05:00,NaN,NaN,en,False,"['blackandwhite', 'blackandwhitephotography', ...",[],744098046,howaldk,Kristian Howald,0,0,0,[],['https://www.instagram.com/p/Br_gsHnhR2_/?utm...,0,https://twitter.com/HowaldK/status/10795026187...,Going home.... #blackandwhite #blackandwhiteph...,18-12-30
86,1079443064094101504,2018-12-30 13:24:16-05:00,NaN,"{'id': '61ecf520fef6ce72', 'url': 'https://api...",en,False,[],[],2618765060,jazzysart,Jazzy 🌻,0,0,6,[],[],0,https://twitter.com/JazzysArt/status/107944306...,when the gotrain is at oakville station i feel...,18-12-30


In [3]:
# NLTK Stop words
import nltk; nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Beatrice
[nltk_data]     Chen\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['go', 'train', 'gotrain', 'bus', 'station', 'gotransit', "transit", 'minute', "service", "toronto", "get", "thank", "work", "use", "make"])
stop_words

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [5]:
#convert tweets to list
data = df.tweet.values.tolist()
#remove "@"
data = [re.sub(r'@\S*\s?', '', tw) for tw in data]

#remove links
data = [re.sub(r'[http|https]*://[a-zA-Z0-9.?/&=:]*', '', tw) for tw in data]

#remove punctuations
data = [re.sub(r'&[-A-Za-z]+;', '', tw) for tw in data]

#remove duplicates
list(set(data))

['',
 'Hey I’m on the 12:13 Lakeshore East train and Kyle is the best at announcing things please have him do a recording and play it on all the trains THANKS. #traintweets',
 'Hey, Monisa Nandi at here is your speedy leaving #Toronto this beautiful early morning.  ',
 'where’s the train. I’m in Whitby and the 18:44 went into reverse and disappeared',
 "it's been 6 business days and I haven't heard back about my claim I submitted: #FG15022307183462 - who should I talk to?",
 "Yay, you've added more weekday trips for Hamilton!!",
 'Wouldn’t it make sense to have crew at the crossing to block traffic instead of having to stop the train, have the crew get off and block traffic for the train to move forward?  less delays then usual and you would move closer to achieving the charter that you never meet!',
 '164 days later. Any news?',
 '#quietzone #drama #gotrain  ',
 'is there any chance of bus cancellations for the go bus from Newmarket to Barrie ?',
 'It’s #InventorsMonth and we’re looki

In [6]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))
data_words[:1]

[['the', 'bernard', 'hellen', 'daily', 'is', 'out', 'stories', 'via']]

In [7]:
#remove stopwords
def remove_stopwords(texts):
    return [[word for word in gensim.utils.simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
data_words_nostops = remove_stopwords(data_words)
data_words_nostops[:1]

[['bernard', 'hellen', 'daily', 'stories', 'via']]

In [8]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

#print(trigram_mod[bigram_mod[data_words[:]]])

In [9]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

data_words_nostops = make_bigrams(data_words_nostops)
data_words_nostops

[['bernard', 'hellen', 'daily', 'stories_via'],
 ['miltonon',
  'good',
  'see',
  'lines',
  'town',
  'lack',
  'parking',
  'bigger',
  'issue',
  'lets',
  'started',
  'working'],
 ['transitprogress',
  'list',
  'almosts',
  'coming',
  'soon',
  'progress',
  'yet',
  'progress',
  'coming',
  'soon'],
 ['rt',
  'miltonon',
  'line',
  'started',
  'today',
  'added',
  'two',
  'new',
  'weekday',
  'trips',
  'details'],
 ['hi',
  'steve',
  'much',
  'already',
  'done',
  'expect',
  'complete',
  'activate',
  'mid'],
 ['new', 'announcement', 'trains', 'minutes', 'earlier', 'yippeeee'],
 ['update',
  'status',
  'ctc',
  'install',
  'along',
  'guelph',
  'junction',
  'silver',
  'london',
  'cc'],
 ['miltonon',
  'line',
  'started',
  'today',
  'added',
  'two',
  'new',
  'weekday',
  'trips',
  'details'],
 ['know', 'working', 'trains', 'matter', 'time'],
 ['live', 'really', 'close', 'one', 'fav', 'parks'],
 ['love', 'trains', 'awesome', 'spot', 'watch', 'rt', 'great

In [10]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

In [11]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'ADV', 'VERB']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

data_lemmatized = lemmatization(data_words_nostops, allowed_postags=['NOUN', 'ADJ', 'ADV', 'VERB'])
data_lemmatized

[['bernard', 'hellen', 'daily'],
 ['miltonon',
  'good',
  'see',
  'line',
  'town',
  'lack',
  'parking',
  'big',
  'issue',
  'let',
  'start',
  'work'],
 ['transitprogress',
  'list',
  'almost',
  'come',
  'soon',
  'progress',
  'yet',
  'progress',
  'come',
  'soon'],
 ['miltonon',
  'line',
  'start',
  'today',
  'add',
  'new',
  'weekday',
  'trip',
  'detail'],
 ['steve', 'much', 'already', 'do', 'expect', 'complete', 'activate', 'mid'],
 ['new', 'announcement', 'train', 'minute', 'early', 'yippeeee'],
 ['update',
  'status',
  'ctc',
  'install',
  'guelph',
  'junction',
  'silver',
  'london',
  'cc'],
 ['miltonon',
  'line',
  'start',
  'today',
  'add',
  'new',
  'weekday',
  'trip',
  'detail'],
 ['know', 'work', 'train', 'matter', 'time'],
 ['live', 'really', 'close', 'fav', 'park'],
 ['love', 'train', 'awesome', 'spot', 'watch', 'great', 'shot'],
 ['great', 'shot'],
 ['monisa', 'nandi', 'speedy', 'leave', 'beautiful', 'early', 'morning'],
 ['co_fare', 'presto

In [12]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('bernard', 1), ('daily', 1), ('hellen', 1)]]

In [13]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=1,
                                           alpha='auto',
                                           per_word_topics=True)
lda_model.print_topics()

[(0,
  '0.134*"work" + 0.093*"happen" + 0.082*"let" + 0.079*"foot" + 0.051*"big" + 0.050*"passenger" + 0.044*"full" + 0.042*"parking" + 0.042*"tweet" + 0.041*"especially"'),
 (1,
  '0.198*"come" + 0.145*"stop" + 0.142*"say" + 0.066*"buy" + 0.044*"track" + 0.042*"soon" + 0.040*"list" + 0.025*"almost" + 0.024*"switch" + 0.021*"finish"'),
 (2,
  '0.126*"today" + 0.119*"line" + 0.117*"good" + 0.114*"trip" + 0.080*"hour" + 0.073*"customer" + 0.050*"start" + 0.048*"add" + 0.040*"commute" + 0.025*"detail"'),
 (3,
  '0.205*"presto" + 0.203*"day" + 0.078*"make" + 0.076*"look" + 0.054*"love" + 0.053*"sign" + 0.045*"ticket" + 0.022*"awesome" + 0.019*"sound" + 0.018*"watch"'),
 (4,
  '0.191*"great" + 0.115*"leave" + 0.113*"much" + 0.092*"give" + 0.077*"home" + 0.064*"st" + 0.047*"early" + 0.031*"later" + 0.023*"catch" + 0.009*"beautiful"'),
 (5,
  '0.117*"late" + 0.096*"help" + 0.073*"min" + 0.070*"nice" + 0.060*"credit" + 0.054*"thing" + 0.045*"cost" + 0.038*"hear" + 0.037*"lol" + 0.034*"traffic"

In [14]:
#visualization
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
9      0.247539  0.305930       1        1  9.267399
2      0.273686 -0.285557       2        1  8.092464
16    -0.017781 -0.001156       3        1  6.690711
3      0.098768  0.007313       4        1  6.424800
12     0.043369  0.002131       5        1  6.197852
5     -0.020853 -0.001294       6        1  5.903948
15    -0.025268 -0.001460       7        1  5.739175
1     -0.003763 -0.000549       8        1  5.290501
10     0.016760  0.000467       9        1  5.014827
18    -0.047844 -0.002189      10        1  4.850783
17    -0.066659 -0.002673      11        1  4.640600
11    -0.065774 -0.002664      12        1  4.462716
7     -0.022711 -0.001336      13        1  4.443998
0     -0.047206 -0.002202      14        1  4.431393
4     -0.036416 -0.001839      15        1  4.340455
14    -0.044540 -0.002097      16        1  3.666887
8     -0.065851 -0.002668      17        1  3.529129
13    -0.069564 -0.002736      18        1  3.250549
6     -0.067439 -0.002694      19        1  2.545216
19    -0.078453 -0.002726      20        1  1.216598, topic_info=        Term         Freq        Total Category  logprob  loglift
52      time  2129.000000  2129.000000  Default  30.0000  30.0000
24       new  1432.000000  1432.000000  Default  29.0000  29.0000
73    presto  1652.000000  1652.000000  Default  28.0000  28.0000
176      day  1637.000000  1637.000000  Default  27.0000  27.0000
228    union  1412.000000  1412.000000  Default  26.0000  26.0000
...      ...          ...          ...      ...      ...      ...
1511  public     0.067372     2.340060  Topic20 -10.0252   0.8614
506     miss     0.067369     2.339853  Topic20 -10.0252   0.8615
537     card     0.067405     2.343570  Topic20 -10.0247   0.8604
463    drive     0.067367     2.339863  Topic20 -10.0253   0.8614
464     want     0.067367     2.340080  Topic20 -10.0253   0.8613

[769 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
85        6  0.986728        able
22        2  0.994867         add
352      11  0.830230  additional
95       18  0.988589         ago
15        8  0.988781      almost
...     ...       ...         ...
218       4  0.978367        wish
200       3  0.990094      wonder
14       14  0.997468        work
121      16  0.997555        year
21       17  0.990131         yet

[263 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 3, 17, 4, 13, 6, 16, 2, 11, 19, 18, 12, 8, 1, 5, 15, 9, 14, 7, 20])

In [15]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  #lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -20.190318734292347

Coherence Score:  0.3576188747365132
